In [8]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from solutions.llm import llm

def movie_qa(prompt):
    graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
    )

    CYPHER_GENERATION_TEMPLATE = """
    You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
    Convert the user's question based on the schema.

    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.

    Fine Tuning:

    For movie titles that begin with "The", move "the" to the end. For example "The 39 Steps" becomes "39 Steps, The" or "the matrix" becomes "Matrix, The".


    Schema:
    {schema}

    Question:
    {question}

    Cypher Query:
    """

    cypher_prompt = PromptTemplate.from_template(CYPHER_GENERATION_TEMPLATE)
    
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        cypher_prompt=cypher_prompt,
        verbose=True
    )

    result = cypher_chain.invoke({prompt})
    return result['result']
    
    
movie_qa("who acted in toy story")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (a:Actor)-[:ACTED_IN]->(m:Movie)
WHERE m.title = 'Toy Story'
RETURN a.name
Full Context:
[{'a.name': 'Jim Varney'}, {'a.name': 'Tim Allen'}, {'a.name': 'Tom Hanks'}, {'a.name': 'Don Rickles'}]

> Finished chain.


'Jim Varney, Tim Allen, Tom Hanks, and Don Rickles acted in Toy Story.'

In [9]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from solutions.llm import llm

def movie_qa(prompt):
    graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
    )

    CYPHER_GENERATION_TEMPLATE = """
    You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
    Convert the user's question based on the schema.

    Instructions:
    Use only the provided relationship types and properties in the schema.
    Do not use any other relationship types or properties that are not provided.
    For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

    If no data is returned, do not attempt to answer the question.
    Only respond to questions that require you to construct a Cypher statement.
    Do not include any explanations or apologies in your responses.

    Examples:

    Find movies and genres:
    MATCH (m:Movie)-[:IN_GENRE]->(g)
    RETURN m.title, g.name

    Schema: {schema}
    Question: {question}
    """

    cypher_generation_prompt = PromptTemplate(
        template=CYPHER_GENERATION_TEMPLATE,
        input_variables=["schema", "question"],
    )
    
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        cypher_prompt=cypher_generation_prompt,
        verbose=True
    )

    result = cypher_chain.invoke({prompt})
    return result['result']

movie_qa("toy story 2")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie) 
WHERE m.title = 'Toy Story 2' 
RETURN m
Full Context:
[{'m': {'languages': ['English', ' Arabic', ' Spanish'], 'year': 1999, 'imdbId': '0120363', 'runtime': 92, 'imdbRating': 7.9, 'movieId': '3114', 'countries': ['USA'], 'imdbVotes': 369499, 'title': 'Toy Story 2', 'url': 'https://themoviedb.org/movie/863', 'revenue': 497366869, 'tmdbId': '863', 'plot': 'When Woody is stolen by a toy collector, Buzz and his friends vow to rescue him, but Woody finds the idea of immortality in a museum tempting.', 'poster': 'https://image.tmdb.org/t/p/w440_and_h660_face/yFWQkz2ynjwsazT6xQiIXEUsyuh.jpg', 'released': '1999-11-24', 'budget': 90000000}}]

> Finished chain.


'"Toy Story 2" is a movie that was released in 1999. The plot revolves around Woody being stolen by a toy collector, leading Buzz and his friends to vow to rescue him. However, Woody finds the idea of immortality in a museum tempting. The movie is in English, Arabic, and Spanish and was produced in the USA. It has a runtime of 92 minutes and an IMDB rating of 7.9 based on 369,499 votes. The movie had a budget of $90,000,000 and generated a revenue of $497,366,869. You can find more information about the movie on this [link](https://themoviedb.org/movie/863).'

In [3]:
from langchain.chains import  LLMChain
from langchain import PromptTemplate
from langchain.prompts import PromptTemplate
from solutions.llm import llm


#assign variables
input_language = 'EN'
output_language = 'TH'

#promptTemplate
summary_templates = """
   You are a translation specialist with 35 years of experience in translating from {input_language} to {output_language}. Given the question {Question}, 
   your task is to translate it into {output_language}. If the question is already in {output_language}, check for any grammatical mistakes and correct them if necessary.
    Caution:
    Provide only the answer, without any explanations.
    """

summary_prompts_template = PromptTemplate(input_variables=['Question','input_language','output_language'], template = summary_templates)

chain = LLMChain(llm=llm, prompt=summary_prompts_template) 

def translate_qa(prompt):
    Answer = chain.run(Question=prompt,input_language=input_language,output_language=output_language)
    print(str(Answer))  
    
translate_qa("how are you today")

คุณเป็นอย่างไรบ้างวันนี้


In [5]:
from langchain_openai import ChatOpenAI
from langchain_community.graphs import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain.prompts import PromptTemplate
from solutions.llm import llm

graph = Neo4jGraph(
    url="bolt://18.207.205.88:7687",
    username="neo4j",
    password="inclinations-odors-laser", 
)

CYPHER_GENERATION_TEMPLATE = """
You are an expert Neo4j Developer translating user questions into Cypher to answer questions about movies and provide recommendations.
Convert the user's question based on the schema.

Instructions:
Use only the provided relationship types and properties in the schema.
Do not use any other relationship types or properties that are not provided.
For movie titles that begin with "The", move "the" to the end, For example "The 39 Steps" becomes "39 Steps, The" or "The Matrix" becomes "Matrix, The".

If no data is returned, do not attempt to answer the question.
Only respond to questions that require you to construct a Cypher statement.
Do not include any explanations or apologies in your responses.

Examples:

Find movies and genres:
MATCH (m:Movie)-[:IN_GENRE]->(g)
RETURN m.title, g.name

Schema: {schema}
Question: {question}
"""

cypher_generation_prompt = PromptTemplate(
    template=CYPHER_GENERATION_TEMPLATE,
    input_variables=["schema", "question"],
)


def movie_qa(prompt):
    cypher_chain = GraphCypherQAChain.from_llm(
        llm=llm,
        graph=graph,
        cypher_prompt=cypher_generation_prompt,
        verbose=True
    )

    result = cypher_chain.run(prompt)
    print(str(result))

movie_qa("who is the actor of toy story")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Movie {title: "Toy Story"})-[:ACTED_IN]-(a:Actor)
RETURN a.name
Full Context:
[{'a.name': 'Jim Varney'}, {'a.name': 'Tim Allen'}, {'a.name': 'Tom Hanks'}, {'a.name': 'Don Rickles'}]

> Finished chain.
The actors of Toy Story are Jim Varney, Tim Allen, Tom Hanks, and Don Rickles.


In [8]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
from solutions.llm import llm


def dmac_qa(prompt):
    """
    Use the  Vector Search Index
    to augment the response from the LLM
    """
    embeddings = OpenAIEmbeddings()
    damac_vectorstore = FAISS.load_local("damac_database", embeddings)
    retriever = damac_vectorstore.as_retriever()
    dmac_qa = RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=retriever,
    verbose=True
    )
    result = dmac_qa.run(prompt)
    return str(result)

prompt = "what is the lease amount"
dmac_qa(prompt)

c:\Users\(Ai)AiSukmoren\.conda\envs\langchain_demo\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(




> Entering new RetrievalQA chain...

> Finished chain.


'The lease amount is 17,000,000 Thailand Bath (seventeen million THB) per year.'

In [9]:
from langchain.tools import Tool
from langchain.agents import AgentExecutor, create_react_agent
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from solutions.llm import llm
from solutions.damac import dmac_qa
from solutions.youtube import youtube
from solutions.graph import movie_qa
from solutions.translator import translate_qa
from langchain.prompts import PromptTemplate



tools = [
    Tool.from_function(
        name="General Chat",
        description="For general chat not covered by other tools",
        func=llm.invoke,
        return_direct=True,
    ),
    Tool.from_function(
        name="Damac agreement",
        description="Provides information about DAMAC agreement",
        func=dmac_qa,
        return_direct=True,
    ),
    Tool.from_function(
        name="Youtube search",
        description="For proving youtube url to user",
        func=youtube.run,
        return_direct=True,
    ),
        Tool.from_function(
        name="Movie serch",
        description="For proving details and information related to movie",
        func=movie_qa,
        return_direct=True,
    ),
        Tool.from_function(
        name="Translator",
        description="Use for when user want to translate the result to TH",
        func=translate_qa,
        return_direct=True,
    ),
        
]

memory = ConversationBufferWindowMemory(
    memory_key="chat_history",
    k=3,
    return_messages=True,
)

agent_prompt = PromptTemplate.from_template(
    """
        You are a lawyer who is specilized in answering ansewer regard to law and regulation decision question.
        Be as helpful as possible and return as much information as possible.

        TOOLS:
        ------

        You have access to the following tools:

        {tools}

        To use a tool, please use the following format:

        ```
        Thought: Do I need to use a tool? Yes
        Action: the action to take, should be one of [{tool_names}]
        Action Input: the {input} to the action
        Observation: the result of the action
        ```

        When you have a response to say to the Human, or if you do not need to use a tool, you MUST use the format:

        ```
        Thought: Do I need to use a tool? No
        Final Answer: [your response here]
        ```

        Begin!

        Previous conversation history:
        {chat_history}

        New input: {input}
        {agent_scratchpad}
        """
)
agent = create_react_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, memory=memory, verbose=True)




def generate_response(prompt):
    """
    Create a handler that calls the Conversational agent
    and returns a response to be rendered in the UI
    """

    response = agent_executor.invoke({"input": prompt})

    return response['output']

while True:
    q = input("> ")
    if q.lower() == "exit":
        print("Exiting...")
        break
    else:
        result = generate_response(q)


        print(result)



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie serch
Action Input: E.TI'm sorry, but I don't know the answer.


> Finished chain.
I'm sorry, but I don't know the answer.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Movie serch
Action Input: who acted in toy storyJim Varney, Tim Allen, Tom Hanks, and Don Rickles acted in Toy Story.


> Finished chain.
Jim Varney, Tim Allen, Tom Hanks, and Don Rickles acted in Toy Story.


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Hello! How can I assist you today?

> Finished chain.
Hello! How can I assist you today?


> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
Final Answer: Toy Story 2 was directed by John Lasseter, co-directed by Ash Brannon and Lee Unkrich.

> Finished chain.
Toy Story 2 was directed by John Lasseter, co-directed by Ash Brannon and Lee Unkrich.


> Entering new AgentExecu

In [9]:
import streamlit as st
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.chains import RetrievalQA
from solutions.llm import llm, embeddings

def kg_qa(prompt):
    """
    Use the Neo4j Vector Search Index
    to augment the response from the LLM
    """
    neo4jvector = Neo4jVector.from_existing_index(
    embeddings,  
    url=st.secrets["NEO4J_URI"],  
    username=st.secrets["NEO4J_USERNAME"],  
    password=st.secrets["NEO4J_PASSWORD"],  
    index_name="moviePlots",  
    node_label="Movie",  
    text_node_property="plot", 
    embedding_node_property="plotEmbedding",  
    retrieval_query="""
    RETURN
        node.plot AS text,
        score,
        {
            title: node.title,
            directors: [ (person)-[:DIRECTED]->(node) | person.name ],
            actors: [ (person)-[r:ACTED_IN]->(node) | [person.name, r.role] ],
            tmdbId: node.tmdbId,
            source: 'https://www.themoviedb.org/movie/'+ node.tmdbId
        } AS metadata
    """,
    )

    retriever = neo4jvector.as_retriever()

    kg_qa = RetrievalQA.from_chain_type(
        llm,  
        chain_type="stuff",  
        retriever=retriever,  
        verbose=True
    )
    # Handle the response
    result = kg_qa.run(prompt)
    return str(result)

kg_qa("what is the plot about jumanji")



> Entering new RetrievalQA chain...

> Finished chain.


'The plot of Jumanji revolves around two kids who find and play a magical board game. By doing so, they inadvertently release a man who has been trapped inside the game for decades. As they continue to play, they encounter various dangers and challenges that can only be stopped by finishing the game. The kids must work together to navigate through the jungle-themed world of Jumanji and find a way to restore order and return everything back to normal.'